In [3]:
import lib.db_connector as db_connector
import pandas as pd

#Set to true if database connection is not available
local = True

if not local:
    db = db_connector.DbConnector(db_name = 'speakql_study', verbose = False)

Initializing DbConnector class for a mysql  connection to speakql_study
Attempting to connect to speakql_study
DBCONNECTOR: Connected to 8.0.32-0buntu0.22.04.1


#### Get existing data from speakql_study db

In [4]:
if not local:
    feature_df = db.do_single_select_query_into_dataframe('select * from feature_usage')
    feature_df.to_excel('./data/df/feature-usage-determination-feature-df.xlsx')
else:
    feature_df = pd.read_excel('./data/df/feature-usage-determination-feature-df.xlsx')
feature_df

,Unnamed: 0,filename,used_synonyms,used_unbundling,used_natural_functions,used_expression_ordering,has_min_kws,used_mod_ordering
0,0,username-participant1_queryid-7_session-39_ste...,False,True,False,False,True,False
1,1,username-participant1_queryid-14_session-39_st...,True,True,True,False,True,False
2,2,username-participant2_queryid-4_session-41_ste...,False,False,False,False,True,False
3,3,username-participant3_queryid-16_session-44_st...,False,False,False,False,True,False
4,4,username-participant3_queryid-4_session-44_ste...,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...
915,915,username-participant23_queryid-5_session-66_st...,False,False,True,False,True,False
916,916,username-participant23_queryid-7_session-66_st...,True,True,False,False,True,False
917,917,username-participant23_queryid-7_session-66_st...,False,False,False,False,True,False
918,918,username-participant23_queryid-9_session-66_st...,False,True,True,True,True,False


#### Get transcript data from SpeakQL study DB:

In [3]:
transcripts = db.do_single_select_query_into_dataframe('select * from whisper_transcripts order by filename')
transcripts

,filename,transcript
0,username-participant1_queryid-1_session-39_ste...,from term table get distinct year.
1,username-participant1_queryid-1_session-39_ste...,Select Distinct ear from turn.
2,username-participant1_queryid-10_session-39_st...,Select building name from building table and ...
3,username-participant1_queryid-10_session-39_st...,Select building name from building table and ...
4,username-participant1_queryid-11_session-39_st...,Select nothing from term table where year is ...
...,...,...
915,username-participant9_queryid-7_session-50_ste...,A as bracket select building ID from room whe...
916,username-participant9_queryid-9_session-50_ste...,Select count of building ID from the room tab...
917,username-participant9_queryid-9_session-50_ste...,A as bracket select building ID from room whe...
918,username-participant9_queryid-9_session-50_ste...,A as bracket select building ID from room whe...


#### Synonym usage determination:

In [4]:
synonyms = [
    "RETRIEVE", "SHOW ME", "DISPLAY", "PRESENT", "FIND", "GET", "WHAT IS", "WHAT ARE", "WHAT IS THE",
    "WHAT ARE THE", "FROM TABLE", "FROM TABLES", "IN TABLE", "IN TABLES", "JOIN TABLE", "BY JOINING",
    "BY JOINING TABLE", "JOINED WITH", "JOIN WITH", "JOINED WITH TABLE", "JOIN WITH TABLE", 
    "BY JOINING WITH TABLE", "BY JOINING WITH"
]

def has_synonym(transcript, synonyms):
    for synonym in synonyms:
        if synonym.lower() in transcript.lower():
            return True
    return False

transcripts['used_synonyms'] = transcripts.apply(
    lambda row: has_synonym(row.transcript, synonyms),
    axis = 1
)

transcripts

,filename,transcript,used_synonyms
0,username-participant1_queryid-1_session-39_ste...,from term table get distinct year.,True
1,username-participant1_queryid-1_session-39_ste...,Select Distinct ear from turn.,False
2,username-participant1_queryid-10_session-39_st...,Select building name from building table and ...,True
3,username-participant1_queryid-10_session-39_st...,Select building name from building table and ...,True
4,username-participant1_queryid-11_session-39_st...,Select nothing from term table where year is ...,False
...,...,...,...
915,username-participant9_queryid-7_session-50_ste...,A as bracket select building ID from room whe...,False
916,username-participant9_queryid-9_session-50_ste...,Select count of building ID from the room tab...,False
917,username-participant9_queryid-9_session-50_ste...,A as bracket select building ID from room whe...,False
918,username-participant9_queryid-9_session-50_ste...,A as bracket select building ID from room whe...,False


#### Unbundling Usage Determination:

In [5]:
transcripts['used_unbundling'] = transcripts.apply(
    lambda row: True if 'and then' in row.transcript.lower() else False,
    axis = 1
)

transcripts['p23'] = transcripts.apply(lambda row: row['filename'] if 'participant23' in row['filename'] else 0, axis = 1)
transcripts.where(transcripts.p23 != 0).dropna(how = 'all')

,filename,transcript,used_synonyms,used_unbundling,p23
613,username-participant23_queryid--1_session-66_s...,"I think I know. Yeah, go ahead and press stop...",0.0,0.0,username-participant23_queryid--1_session-66_s...
614,username-participant23_queryid-1_session-66_st...,Get a distinct year from term.,1.0,0.0,username-participant23_queryid-1_session-66_st...
615,username-participant23_queryid-1_session-66_st...,Select Distinct Ear from Turn,0.0,0.0,username-participant23_queryid-1_session-66_st...
616,username-participant23_queryid-10_session-66_s...,Select sum of wheelchair spaces from room R a...,0.0,1.0,username-participant23_queryid-10_session-66_s...
617,username-participant23_queryid-10_session-66_s...,"Select building name, sum, wheel chair spaces...",0.0,0.0,username-participant23_queryid-10_session-66_s...
618,username-participant23_queryid-11_session-66_s...,Select title from course C and then select no...,0.0,1.0,username-participant23_queryid-11_session-66_s...
619,username-participant23_queryid-11_session-66_s...,"Select title from course C, join course C on ...",0.0,0.0,username-participant23_queryid-11_session-66_s...
620,username-participant23_queryid-11_session-66_s...,"Select title from course C, join course offer...",0.0,0.0,username-participant23_queryid-11_session-66_s...
621,username-participant23_queryid-12_session-66_s...,Select department name from department D and ...,0.0,1.0,username-participant23_queryid-12_session-66_s...
622,username-participant23_queryid-12_session-66_s...,"Select Department Name, Count, Parenthesis, D...",0.0,0.0,username-participant23_queryid-12_session-66_s...


#### Natural Function Usage Determination

In [6]:
function_patterns = [
    "the count", "count of", "the sum", "sum of", "the average", "average of"
]

function_keywords = ["count", "sum", "average"]

def used_nat_func(transcript):
    # Check if used the or of keywords and return true if used
    for pattern in function_patterns:
        if pattern in transcript:
            return True
    # Check if functions used without parentheses and return true if used
    no_parens = False
    for kw in function_keywords:
        if kw in transcript:
            no_parens = True
            for p in ['parenthesis', 'parentheses']:
                if p in transcript:
                    no_parens = False
    
    return no_parens
    

transcripts['used_natural_functions'] = transcripts.apply(
    lambda row: used_nat_func(row.transcript),
    axis = 1
)



#### Alternate Ordering Usage Determination
Run the following cell first before running either expression or modifier reordering determination

In [7]:
from lib.SpeakqlKeywords import SpeakQlKeywords 
spkw = SpeakQlKeywords()

In [8]:
def used_alternate_ordering(transcript, spkw):
    queries = transcript.upper().split('AND THEN')
    from_position = -1
    select_position = -1
    where_position = -1
    used_alt = False
    
    for query in queries:
        for kw in spkw.from_kw:
            frm_loc = query.find(kw)
            if frm_loc > from_position:
                from_position = frm_loc
    
        for kw in spkw.select_kw:
            sel_loc = query.find(kw)
            if sel_loc > select_position:
                select_position = sel_loc
                
        where_loc = query.find('WHERE')
        if where_loc > where_position:
            where_position = where_loc
            
    return (
           (from_position < select_position) 
        or (((from_position > where_position) or (select_position > where_position)) and where_position > -1)
    )

transcripts['used_expression_ordering'] = transcripts.apply(
    lambda row: used_alternate_ordering(row.transcript, spkw),
    axis = 1
)

transcripts



,filename,transcript,used_synonyms,used_unbundling,p23,used_natural_functions,used_expression_ordering
0,username-participant1_queryid-1_session-39_ste...,from term table get distinct year.,True,False,0,False,False
1,username-participant1_queryid-1_session-39_ste...,Select Distinct ear from turn.,False,False,0,False,False
2,username-participant1_queryid-10_session-39_st...,Select building name from building table and ...,True,True,0,True,False
3,username-participant1_queryid-10_session-39_st...,Select building name from building table and ...,True,True,0,True,False
4,username-participant1_queryid-11_session-39_st...,Select nothing from term table where year is ...,False,True,0,False,False
...,...,...,...,...,...,...,...
915,username-participant9_queryid-7_session-50_ste...,A as bracket select building ID from room whe...,False,False,0,False,False
916,username-participant9_queryid-9_session-50_ste...,Select count of building ID from the room tab...,False,False,0,True,False
917,username-participant9_queryid-9_session-50_ste...,A as bracket select building ID from room whe...,False,False,0,True,False
918,username-participant9_queryid-9_session-50_ste...,A as bracket select building ID from room whe...,False,False,0,True,False


In [9]:
def used_modifier_ordering(transcript):
    NOT_IN_QUERY = 1000000
    
    queries = transcript.upper().split("AND THEN")
    
    for q in queries:
        gb_pos = q.find("GROUP BY")
        if gb_pos == -1:
            gb_pos = NOT_IN_QUERY
        h_pos = q.find("HAVING")
        if h_pos == -1:
            h_pos = NOT_IN_QUERY
        ob_pos = q.find("ORDER BY")
        if ob_pos == -1:
            ob_pos = NOT_IN_QUERY
        l_pos = q.find("LIMIT")
        if l_pos == -1:
            l_pos = NOT_IN_QUERY
        
    if gb_pos != NOT_IN_QUERY and (gb_pos > h_pos or gb_pos > ob_pos or gb_pos > l_pos):
        return True
    
    if h_pos != NOT_IN_QUERY and (h_pos > ob_pos or h_pos > l_pos):
        return True
    
    if ob_pos != NOT_IN_QUERY and (ob_pos > l_pos):
        return True
    
    return False
            
transcripts['used_mod_ordering'] = transcripts.apply(
    lambda row: used_modifier_ordering(row.transcript),
    axis = 1
)

transcripts
    

,filename,transcript,used_synonyms,used_unbundling,p23,used_natural_functions,used_expression_ordering,used_mod_ordering
0,username-participant1_queryid-1_session-39_ste...,from term table get distinct year.,True,False,0,False,False,False
1,username-participant1_queryid-1_session-39_ste...,Select Distinct ear from turn.,False,False,0,False,False,False
2,username-participant1_queryid-10_session-39_st...,Select building name from building table and ...,True,True,0,True,False,False
3,username-participant1_queryid-10_session-39_st...,Select building name from building table and ...,True,True,0,True,False,False
4,username-participant1_queryid-11_session-39_st...,Select nothing from term table where year is ...,False,True,0,False,False,False
...,...,...,...,...,...,...,...,...
915,username-participant9_queryid-7_session-50_ste...,A as bracket select building ID from room whe...,False,False,0,False,False,False
916,username-participant9_queryid-9_session-50_ste...,Select count of building ID from the room tab...,False,False,0,True,False,False
917,username-participant9_queryid-9_session-50_ste...,A as bracket select building ID from room whe...,False,False,0,True,False,False
918,username-participant9_queryid-9_session-50_ste...,A as bracket select building ID from room whe...,False,False,0,True,False,False


In [10]:
def min_req_kws(transcript, spkw):
    has_from = False
    has_select = False
    query = transcript.upper()
    for kw in spkw.from_kw:
        if kw in query:
            has_from = True
    for kw in spkw.select_kw:
        if kw in query:
            has_select = True
            
    return has_select and has_from

transcripts['has_min_kws'] = transcripts.apply(
    lambda row: min_req_kws(row.transcript, spkw),
    axis = 1
)

transcripts


,filename,transcript,used_synonyms,used_unbundling,p23,used_natural_functions,used_expression_ordering,used_mod_ordering,has_min_kws
0,username-participant1_queryid-1_session-39_ste...,from term table get distinct year.,True,False,0,False,False,False,True
1,username-participant1_queryid-1_session-39_ste...,Select Distinct ear from turn.,False,False,0,False,False,False,True
2,username-participant1_queryid-10_session-39_st...,Select building name from building table and ...,True,True,0,True,False,False,True
3,username-participant1_queryid-10_session-39_st...,Select building name from building table and ...,True,True,0,True,False,False,True
4,username-participant1_queryid-11_session-39_st...,Select nothing from term table where year is ...,False,True,0,False,False,False,True
...,...,...,...,...,...,...,...,...,...
915,username-participant9_queryid-7_session-50_ste...,A as bracket select building ID from room whe...,False,False,0,False,False,False,True
916,username-participant9_queryid-9_session-50_ste...,Select count of building ID from the room tab...,False,False,0,True,False,False,True
917,username-participant9_queryid-9_session-50_ste...,A as bracket select building ID from room whe...,False,False,0,True,False,False,True
918,username-participant9_queryid-9_session-50_ste...,A as bracket select building ID from room whe...,False,False,0,True,False,False,True


## Data Export Jobs

#### Export to Excel:

In [11]:
transcripts.to_excel("./features_used.xlsx")

#### Insert into study database (Only run if new data is processed)

In [12]:
query = """insert into feature_usage(
filename, used_synonyms, used_unbundling, used_natural_functions, used_expression_ordering, has_min_kws, used_mod_ordering
) values (
'{}', '{}', '{}', '{}', '{}', '{}', '{}'
)"""

existing_rows = db.do_single_select_query_into_dataframe("select * from feature_usage")
existing_rows

for row in transcripts.itertuples():
    if row.filename not in existing_rows.filename.to_list():
        insert_q = query.format(
                row.filename, 
                row.used_synonyms, 
                row.used_unbundling, 
                row.used_natural_functions, 
                row.used_expression_ordering, 
                row.has_min_kws, 
                row.used_mod_ordering)
        print(insert_q)
    
        db.do_single_insert_query_into_dataframe(
            insert_q
        )